# This Notebook illustrates how to run an efficient multilayer QNN, in pytorch, with feature ranking



## Import Libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ,MinMaxScaler , Normalizer
from torch.nn.functional import normalize
from torch.utils.data import Dataset, DataLoader , TensorDataset
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve
import numpy as np
import pandas as pd

## Import taiwan data

In [3]:
!wget http://archive.ics.uci.edu/static/public/350/default+of+credit+card+clients.zip
!unzip default*

--2025-11-19 14:18:53--  http://archive.ics.uci.edu/static/public/350/default+of+credit+card+clients.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘default+of+credit+card+clients.zip.1’

default+of+credit+c     [   <=>              ]   5.28M  11.0MB/s    in 0.5s    

2025-11-19 14:18:54 (11.0 MB/s) - ‘default+of+credit+card+clients.zip.1’ saved [5539494]

Archive:  default of credit card clients.xls
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
note:  default of credit card clients.xls may be a plain executable, not an archive
unzip:  cannot find zipfile directory in one of default of cred

In [4]:
# Load the file directly into a NumPy array
data = pd.read_excel("/content/default of credit card clients.xls", header=1)
data = data.drop(columns=["ID"])

# We need to split and scale the data



In [5]:
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
np.random.seed(32)

X = data.drop(columns=["default payment next month"])  # All columns except target
y = data["default payment next month"]  # Target column

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.3333, random_state=42, stratify=y_temp)

pt = PowerTransformer(method="yeo-johnson")
X_train = pt.fit_transform(X_train)
X_val = pt.transform(X_val)
X_test = pt.transform(X_test)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

## Define the QNN class

In [6]:
import torch
import torch.nn as nn
import math
from functools import reduce

def get_SU_d_gens(d):
    generators = []

    # symmetric generators
    for j in range(d-1):
        for k in range(j+1, d):
            gen = torch.zeros(d, d, dtype=torch.complex128)
            gen[j, k] = 1
            gen[k, j] = 1
            generators.append(gen)

    # antisymmetric generators
    for j in range(d-1):
        for k in range(j+1, d):
            gen = torch.zeros(d, d, dtype=torch.complex128)
            gen[j, k] = -1j
            gen[k, j] = 1j
            generators.append(gen)

    # diagonal generators
    for l in range(1, d):
        gen = torch.zeros(d, d, dtype=torch.complex128)
        constant = math.sqrt(2/((l * (l+1))))
        for j in range(l):
            gen[j, j] = constant
        gen[l, l] = -constant*l
        generators.append(gen)

    return generators

class QNN(nn.Module):
    def __init__(self, qudit_dim, layers):
        torch.manual_seed(1234)
        super(QNN, self).__init__()
        self.d = qudit_dim
        self.layers = layers
        self.gens = torch.zeros(self.d**2 - 1 - 1, 1, self.d, self.d, dtype=torch.complex128)
        _gens = get_SU_d_gens(self.d)
        for i in range(self.d**2 - 1 - 1):
            self.gens[i,:,:,:] = _gens[i]
        self.qudit0 = torch.zeros(self.d, dtype=torch.complex128)
        self.qudit0[0] = 1
        self.num_gens = 23 #len(self.gens)
        self.num_params = self.num_gens * self.layers
        self.weights = nn.Parameter(torch.rand(self.num_gens, self.layers, dtype=torch.float64))

    def forward(self, x):
            weights_m = self.weights
            thetas = x.unsqueeze(1) * weights_m.t().unsqueeze(0)
            thetas = 2 * torch.atan(2*thetas)
            gensWithWeights = thetas[:, :, :, None, None] * self.gens[None, None, :, 0, :, :]
            hermitianMatrix = gensWithWeights.sum(dim=2)
            unitaryMatrix = torch.matrix_exp(1j * hermitianMatrix)
            # finalUnitary = torch.prod(unitaryMatrix, dim=1)
            finalUnitary = reduce(torch.bmm, [unitaryMatrix[:,i] for i in range(unitaryMatrix.shape[1])])
            results = torch.matmul(finalUnitary, self.qudit0)
            probalityVector = torch.abs(results)**2
            sums = torch.sum(probalityVector, dim=1).view(-1, 1)
            return probalityVector.real / sums.real


## Training Loop

In [7]:
num_layers = 8
qnn_model = QNN(5, num_layers)


# Define the loss function and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(qnn_model.parameters(), lr=0.003)


# Define the number of epochs
num_epochs = 100

# Create DataLoader for batch processing
batch_size = 4


labels = np.eye(5)[y_train.values]


dataset_train = TensorDataset(torch.tensor(X_train_scaled) , torch.tensor(labels) )
data_loader_train = DataLoader(dataset_train,batch_size= batch_size,shuffle=True)

In [8]:
labels_test = np.eye(5)[y_test.values]

dataset_test = TensorDataset(torch.tensor(X_test_scaled) , torch.tensor(labels_test) )
data_loader_test = DataLoader(dataset_test,batch_size= batch_size,shuffle=True)

In [9]:
labels_val = np.eye(5)[y_val.values]

dataset_val = TensorDataset(torch.tensor(X_val_scaled), torch.tensor(labels_val))
data_loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

# Ridge Classification

In [10]:
# Set the L1 regularization strength (lambda)
lambda_l1 = 0.00001

training_threshold = 0.0002

prev_loss = torch.inf
no_decr = 0
epoch_threshold = 5

# Training loop
for epoch in range(num_epochs):
    qnn_model.train()
    total_loss = 0.0
    for inputs, labels in data_loader_train:

        threshold = 3.14

        optimizer.zero_grad()

        # Forward pass
        outputs = qnn_model(inputs)

        # Compute the primary loss
        labels = labels.to(float)
        loss = criterion(outputs, labels)

        # Add L1 regularization loss
        l1_loss = 0.0
        for param in qnn_model.parameters():
            l1_loss += torch.sum(torch.abs(param))  # Sum of absolute values of weights
        loss += lambda_l1 * l1_loss  # Add L1 penalty to the primary loss

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        total_loss += loss.item()

    # Print the average loss for each epoch
    average_loss = total_loss / len(data_loader_train)
    # print(f'Epoch {epoch + 1:3d}/{num_epochs}, Loss: {average_loss}')

    qnn_model.eval()
    val_loss_total = 0.0

    with torch.no_grad():
        for val_inputs, val_labels in data_loader_val:
            val_outputs = qnn_model(val_inputs)
            val_labels = val_labels.to(torch.float)
            val_loss = criterion(val_outputs, val_labels)
            val_loss_total += val_loss.item()

    average_val_loss = val_loss_total / len(data_loader_val)

    if average_val_loss + training_threshold > prev_loss:
        no_decr += 1
        if no_decr == epoch_threshold:
            qnn_model.load_state_dict(torch.load('model_weights.pth'))
            break
    else:
        torch.save(qnn_model.state_dict(), 'model_weights.pth')
        no_decr = 0
        prev_loss = average_val_loss


    print(f'Epoch {epoch + 1:3d}/{num_epochs}, '
          f'Train Loss: {average_loss:.4f}, '
          f'Val Loss: {average_val_loss:.4f} '
          f'Inc Streak: {no_decr}')



Epoch   1/100, Train Loss: 1.2333, Val Loss: 1.1495 Inc Streak: 0
Epoch   2/100, Train Loss: 1.1359, Val Loss: 1.1247 Inc Streak: 0
Epoch   3/100, Train Loss: 1.1200, Val Loss: 1.1119 Inc Streak: 0
Epoch   4/100, Train Loss: 1.1132, Val Loss: 1.1065 Inc Streak: 0
Epoch   5/100, Train Loss: 1.1099, Val Loss: 1.1080 Inc Streak: 1
Epoch   6/100, Train Loss: 1.1068, Val Loss: 1.1131 Inc Streak: 2
Epoch   7/100, Train Loss: 1.1056, Val Loss: 1.0999 Inc Streak: 0
Epoch   8/100, Train Loss: 1.1033, Val Loss: 1.0985 Inc Streak: 0
Epoch   9/100, Train Loss: 1.1022, Val Loss: 1.1018 Inc Streak: 1
Epoch  10/100, Train Loss: 1.1012, Val Loss: 1.1014 Inc Streak: 2
Epoch  11/100, Train Loss: 1.1009, Val Loss: 1.0961 Inc Streak: 0
Epoch  12/100, Train Loss: 1.0997, Val Loss: 1.0998 Inc Streak: 1
Epoch  13/100, Train Loss: 1.0987, Val Loss: 1.0951 Inc Streak: 0
Epoch  14/100, Train Loss: 1.0976, Val Loss: 1.0949 Inc Streak: 0
Epoch  15/100, Train Loss: 1.0963, Val Loss: 1.0943 Inc Streak: 0
Epoch  16/

## Get classification results


In [11]:
qnn_model.eval()  # Set the model to evaluation mode

all_predictions = []  # To store all predictions
all_labels = []       # To store all actual labels

with torch.no_grad():
    for inputs, labels_test in data_loader_test:
        outputs = qnn_model(inputs)
        _, predicted = torch.max(outputs, 1)  # Get the predicted class indices

        # Collect predictions and labels
        all_predictions.extend(predicted.cpu().numpy())  # Convert to numpy array and append
        all_labels.extend(labels_test.argmax(dim=1).cpu().numpy())  # Convert labels to numpy and append

In [12]:
predictions_series = pd.Series(all_predictions)
labels_series = pd.Series(all_labels)

In [13]:
predictions_series = predictions_series.clip(upper=1)

In [14]:
report = classification_report(labels_series, predictions_series)
print("\nClassification Report:\n")
print(report)


Classification Report:

              precision    recall  f1-score   support

           0       0.84      0.95      0.89      5841
           1       0.67      0.34      0.45      1659

    accuracy                           0.82      7500
   macro avg       0.75      0.65      0.67      7500
weighted avg       0.80      0.82      0.79      7500



## Print Feature Importance

In [15]:
# Prepare a dictionary to store parameter row sums
row_importance_dict = {}

# Iterate through model parameters
for name, param in qnn_model.named_parameters():
    # Compute the sum of absolute values for each row
    row_sums = param.data.abs().sum(dim=1).tolist()  # Sum absolute values across columns
    row_importance_dict[name] = row_sums

# Convert the dictionary to a Pandas DataFrame for better readability
importance_df = pd.DataFrame([
    {"Layer": name, "Row": i, "Importance (Abs Sum)": row_sum}
    for name, row_sums in row_importance_dict.items()
    for i, row_sum in enumerate(row_sums)
])

# Sort by importance
importance_df = importance_df.sort_values(by="Importance (Abs Sum)", ascending=False)

# Display the importance hierarchy
print("\nRow Importance Hierarchy:\n")
print(importance_df)


Row Importance Hierarchy:

      Layer  Row  Importance (Abs Sum)
12  weights   12             14.610908
11  weights   11             11.822640
13  weights   13             10.382094
5   weights    5              7.002505
16  weights   16              5.702730
14  weights   14              4.097071
15  weights   15              3.203190
6   weights    6              3.100895
9   weights    9              2.120900
7   weights    7              1.572244
10  weights   10              1.510757
17  weights   17              0.856733
8   weights    8              0.802849
18  weights   18              0.544407
0   weights    0              0.419195
22  weights   22              0.406064
2   weights    2              0.359030
3   weights    3              0.330887
19  weights   19              0.288909
20  weights   20              0.265638
21  weights   21              0.208332
4   weights    4              0.112168
1   weights    1              0.046573
